In [1]:
print(5)

5


In [2]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
url = "https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=0000000000&period2=2000000000&interval=1d&events=history&includeAdjustedClose=true"
response = requests.get(url)
response.text
df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
df_plot = df[{'Open','High','Low','Close'}]

In [3]:
df_plot.tail()

,Low,High,Open,Close
Date,,,,
2021-03-13,56217.972656,61683.863281,57343.371094,61243.085938
2021-03-14,59302.316406,61597.917969,61221.132813,59302.316406
2021-03-15,55393.164063,60540.992188,59267.429688,55907.199219
2021-03-16,53555.027344,56833.179688,55840.785156,56804.902344
2021-03-17,55423.562500,57113.089844,55750.421875,55892.796875


In [4]:
#計算ma
def moving_average(data,periods):
    return data['Close'].rolling(periods).mean()

In [5]:
df_copy = df.copy()
df_copy['min'] = df_copy['Low'].rolling(9).min()
df_copy['max'] = df_copy['High'].rolling(9).min()
df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
df_copy = df_copy.dropna()
K_list = [50]
for index,rsv in enumerate(list(df_copy['RSV'])):
    K_yesterday = K_list[index]
    K_today = 2/3 * K_yesterday + 1/3 *rsv
    K_list.append(K_today)
df_copy['K'] = K_list[1:]
D_list = [50]
for index,K in enumerate(list(df_copy['K'])):
    D_yesterday = D_list[index]
    D_today = 2/3 * D_yesterday + 1/3 *K
    D_list.append(D_today)
df_copy['D'] = D_list[1:]

In [6]:
df_copy.tail()

,Open,High,Low,Close,Adj Close,Volume,min,max,RSV,K,D
Date,,,,,,,,,,,
2021-03-13,57343.371094,61683.863281,56217.972656,61243.085938,61243.085938,6.066983e+10,46542.515625,49147.218750,5.643856,4.234079,3.316039
2021-03-14,61221.132813,61597.917969,59302.316406,59302.316406,59302.316406,4.390123e+10,47257.527344,49147.218750,6.373945,4.947368,3.859815
2021-03-15,59267.429688,60540.992188,55393.164063,55907.199219,55907.199219,6.641937e+10,48918.679688,51384.367188,2.834309,4.243015,3.987548
2021-03-16,55840.785156,56833.179688,53555.027344,56804.902344,56804.902344,5.974980e+10,49506.054688,52314.070313,2.599290,3.695107,3.890068
2021-03-17,55750.421875,57113.089844,55423.562500,55892.796875,55892.796875,5.031823e+10,51981.832031,54824.117188,1.375993,2.922069,3.567401


In [7]:
#計算KD值
'''
step1 計算RSV:(今天的收盤價 - 最近9天的最低價)/(最近9天的最高價 - 最近9天的最低價)
step2 計算K值: K = 2/3 * (昨天的K值) + 1/3 * (今日RSV)
step3 計算D值: K = 2/3 * (昨天的D值) + 1/3 * (今日的K值)
'''
def KD(data):
    df_copy = data.copy()
    df_copy['min'] = df_copy['Low'].rolling(9).min()
    df_copy['max'] = df_copy['High'].rolling(9).min()
    df_copy['RSV'] = (df_copy['Close'] - df_copy['min']) / (df_copy['max'] - df_copy['min'])
    df_copy = df_copy.dropna()
    K_list = [50]
    for index,rsv in enumerate(list(df_copy['RSV'])):
        K_yesterday = K_list[index]
        K_today = 2/3 * K_yesterday + 1/3 *rsv
        K_list.append(K_today)
    df_copy['K'] = K_list[1:]
    D_list = [50]
    for index,K in enumerate(list(df_copy['K'])):
        D_yesterday = D_list[index]
        D_today = 2/3 * D_yesterday + 1/3 *K
        D_list.append(D_today)
    df_copy['D'] = D_list[1:]
    use_df = pd.merge(data,df_copy[['K','D']],left_index=True,right_index=True,how='left')
    return use_df

In [8]:
KD(df).tail(10)

,Open,High,Low,Close,Adj Close,Volume,K,D
Date,,,,,,,,
2021-03-08,51174.117188,52314.070313,49506.054688,52246.523438,52246.523438,4.859743e+10,2.082265,1.783336
2021-03-09,52272.968750,54824.117188,51981.832031,54824.117188,54824.117188,5.091223e+10,2.190816,1.919163
2021-03-10,54824.011719,57258.253906,53290.890625,56008.550781,56008.550781,5.729558e+10,2.671947,2.170091
2021-03-11,55963.179688,58091.062500,54484.593750,57805.121094,57805.121094,5.677234e+10,3.222615,2.520932
2021-03-12,57821.218750,57996.621094,55376.648438,57332.089844,57332.089844,5.568994e+10,3.529191,2.857018
2021-03-13,57343.371094,61683.863281,56217.972656,61243.085938,61243.085938,6.066983e+10,4.234079,3.316039
2021-03-14,61221.132813,61597.917969,59302.316406,59302.316406,59302.316406,4.390123e+10,4.947368,3.859815
2021-03-15,59267.429688,60540.992188,55393.164063,55907.199219,55907.199219,6.641937e+10,4.243015,3.987548
2021-03-16,55840.785156,56833.179688,53555.027344,56804.902344,56804.902344,5.974980e+10,3.695107,3.890068
